In [1]:
def x2text(seq, onehot=False):
    if onehot:
        idx = np.argmax(seq.cpu(), axis=2)
    else:
        idx = seq.cpu().numpy()
    text = []
    for i in range(seq.shape[1]):
        text.append(VOCAB_LOOKUP[idx[:, i]])
    return text
    

def y2text(manyhot_label):
    labels = np.array(np.load("./data/labels.npy"))
    all_labels = []    
    for i in range(manyhot_label.shape[0]):
        assert manyhot_label[i].sum() > 0
        all_labels.append(labels[manyhot_label[i].astype(bool)])
    return(all_labels)


def error_analysis(data, m, onehot=False):
    for x, y in data:
        pred = m(x)
        batch_size = x.shape[1]
        y_pred = (torch.sigmoid(pred) > 0.5).int().cpu().numpy()
        for i in range(batch_size):
            if y_pred[i, :].sum() == 0:
                y_pred[i, np.argmax(y_pred[i, :])] = 1
        
        correct = np.all(y_pred==y, axis=1)
        print("total correct: {} out of {}".format(correct.sum(), batch_size))
        
        incorrect_idx = np.arange(batch_size)[(1-correct).astype(bool)]

        x_text = x2text(x, onehot=onehot)
        y_label_true = y2text(y)
        y_label_pred = y2text(y_pred)
        
        for i in incorrect_idx:
#             break
            print(x_text[i])
            print("--------------")
            print("true", y_label_true[i])
            print("pred", y_label_pred[i])
            print("correct?", correct[i])
            print()